In [ ]:
! pip install transformers[torch]
! pip install accelerate
! pip install datasets
! pip install trl
# Docs: https://huggingface.co/docs/trl/en/sft_trainer (version=V0.19.0)
# https://www.datacamp.com/tutorial/fine-tuning-qwen3

In [ ]:

SYSTEM_PROMPT = '''You are a language learning evaluator assessing the complexity of an English sentence given its context.

Rubric:
1 (A1) – Very basic words and phrases; simple self-introduction; minimal grammar.
2 (A2) – Simple sentences; familiar everyday expressions; limited range.
3 (B1) – Can write or speak in connected sentences about familiar topics; some errors.
4 (B2) – Generally fluent; can discuss abstract topics; good grammar control.
5 (C1) – Flexible, natural use of language; few errors; advanced vocabulary.
6 (C2) – Near-native mastery; precise, nuanced expression; fully natural flow.

Please give a rating between 1-6 following the rubric above.
'''

PROMPT_TEMPLATE = '''

Context: {{ context }}
Sentence: {{ sentence }}
Rating (1-6):
'''



In [9]:
import json
from datasets import load_dataset

dataset = load_dataset("dataset/readme")
print(dataset)


Generating train split: 3243 examples [00:00, 91690.05 examples/s]
Generating validation split: 995 examples [00:00, 108853.45 examples/s]
Generating test split: 995 examples [00:00, 114825.49 examples/s]

DatasetDict({
    train: Dataset({
        features: ['Domain', 'Sub-domain', 'Paragraph', 'Context', 'Sentence', 'Rating'],
        num_rows: 3243
    })
    validation: Dataset({
        features: ['Domain', 'Sub-domain', 'Paragraph', 'Context', 'Sentence', 'Rating'],
        num_rows: 995
    })
    test: Dataset({
        features: ['Domain', 'Sub-domain', 'Paragraph', 'Context', 'Sentence', 'Rating'],
        num_rows: 995
    })
})


In [10]:

from jinja2 import Template

JINJA_PROMPT_TEMPLATE = Template(PROMPT_TEMPLATE)

def preprocess_function_wo_chat_template(example):
    prompt = SYSTEM_PROMPT + JINJA_PROMPT_TEMPLATE.render(context=example['Context'], sentence=example['Sentence'])
    return {
        "prompt": prompt,
        "completion": str(example["Rating"]),
    }


def preprocess_function_w_chat_template(example):
    prompt = JINJA_PROMPT_TEMPLATE.render(context=example['Context'], sentence=example['Sentence'])
    return {
        "prompt": [{"role": "system", "content": SYSTEM_PROMPT},
                   {"role": "user", "content":  prompt}],
        "completion": [{"role": "assistant", "content": str(example["Rating"])}]
    }

dataset = dataset.map(preprocess_function_wo_chat_template)
item = next(iter(dataset["train"]))
print(item['prompt'])
print(item['completion'])

Map: 100%|██████████| 995/995 [00:00<00:00, 11413.74 examples/s]

You are a language learning evaluator assessing the complexity of an English sentence given its context.

Rubric:
1 (A1) – Very basic words and phrases; simple self-introduction; minimal grammar.
2 (A2) – Simple sentences; familiar everyday expressions; limited range.
3 (B1) – Can write or speak in connected sentences about familiar topics; some errors.
4 (B2) – Generally fluent; can discuss abstract topics; good grammar control.
5 (C1) – Flexible, natural use of language; few errors; advanced vocabulary.
6 (C2) – Near-native mastery; precise, nuanced expression; fully natural flow.

Please give a rating between 1-6 following the rubric above.


Context: In economics, the firm’s objective is assumed to be to maximize profits. Firms with  market power do this by capturing consumer surplus, and converting it to producer  surplus.
Sentence: In Figure 4.1, a monopoly finds the profit -maximizing price and quantity by  setting MR equal to MC.
Rating (1-6):
4.0


In [17]:
import transformers
import torch

model = transformers.AutoModelForCausalLM.from_pretrained(
                                            "Qwen/Qwen3-4B",
                                            dtype=torch.bfloat16,
                                            device_map="auto")
tokenizer = transformers.AutoTokenizer.from_pretrained("Qwen/Qwen3-4B")

ValueError: Could not find Qwen3ForCausalLM neither in <module 'transformers.models.qwen3' from '/home/ubuntu/.local/lib/python3.10/site-packages/transformers/models/qwen3/__init__.py'> nor in <module 'transformers' from '/home/ubuntu/.local/lib/python3.10/site-packages/transformers/__init__.py'>!

In [13]:

from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(
        max_length=1024,
        output_dir=f"downloads/models/sft_hypertune_qwen3/{str(lr)}_{str(batch_size)}",
        packing=False,
        per_device_train_batch_size=batch_size,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=10,
        learning_rate=lr,
        save_total_limit=1,
        metric_for_best_model="eval_loss",
        load_best_model_at_end=True,
    )

trainer = SFTTrainer(
        model,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        args=training_args,
    )

trainer.train()


RuntimeError: Failed to import trl.trainer.sft_trainer because of the following error (look up to see its traceback):
'type' object is not subscriptable